# Segment Images

In [1]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from loguru import logger
import spatialdata as sd

from plex_pipe.utils.config_loaders import load_analysis_settings
from plex_pipe.processors import build_processor
from plex_pipe.processors.controller import ResourceBuildingController

## Read in config

In [2]:
# load analysis configuration
config_path = r'../examples/example_pipeline_config.yaml'

config = load_analysis_settings(config_path)

# specifies if a resource should be overwritten if it already exists. 
# If False, the pipeline will throw an error if it tries to overwrite an existing resource.
# If True, the pipeline will overwrite existing resources. Use with caution!
OVERWRITE_FLAG = True

2026-02-18 17:30:31.977 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:49 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


## Define the logger

In [3]:
log_file = config.log_dir_path / f"rois_segmentation_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

## Define ROIs for processing

In [4]:
roi_dir = config.analysis_dir / 'rois'
path_list = [roi_dir / f for f in os.listdir(roi_dir)]
path_list.sort()
path_list

[WindowsPath('../examples/output/sample_analysis/rois/ROI_000.zarr'),
 WindowsPath('../examples/output/sample_analysis/rois/ROI_001.zarr')]

In [ ]:
# # create a subset of rois (optional)
# path_list = path_list[:1]

## Setup processors

In [5]:
# setup builders of additional data elements

if getattr(config,'additional_elements',None):
    
    builders_list = []

    for builder_settings in config.additional_elements:
        
        params = dict(getattr(builder_settings,'parameters',None)) or {}

        builder = build_processor(builder_settings.category, builder_settings.type, **params) 
        
        builder_controller = ResourceBuildingController(builder=builder, 
                                            input_names=builder_settings.input, 
                                            output_names=builder_settings.output, 
                                            keep=builder_settings.keep, 
                                            overwrite=OVERWRITE_FLAG,
                                            pyramid_levels=config.sdata_storage.max_pyramid_level,
                                            downscale = config.sdata_storage.downscale,
                                            chunk_size = config.sdata_storage.chunk_size,
                                            )
        
        logger.info(f"Image processor of type '{builder_settings.type}' for image '{builder_settings.input}' has been created.")

        builders_list.append(builder_controller)

else:
    builders_list = []
    logger.info("No resource builders specified.")

2026-02-18 17:30:40.792 | INFO     | __main__:<module>:23 - Image processor of type 'normalize' for image 'DAPI' has been created.
2026-02-18 17:30:40.793 | INFO     | __main__:<module>:23 - Image processor of type 'normalize' for image 'NaKATPase' has been created.
Model fluorescence_nuclei_and_cells version 0.1.1 already downloaded in d:\plex-pipe\.venv\Lib\site-packages\instanseg\utils\../bioimageio_models/, loading
Requesting default device: cuda
2026-02-18 17:30:56.179 | INFO     | __main__:<module>:23 - Image processor of type 'instanseg' for image '['DAPI_norm', 'NaKATPase_norm']' has been created.
2026-02-18 17:30:56.179 | INFO     | __main__:<module>:23 - Image processor of type 'ring' for image '['instanseg_nucleus']' has been created.
2026-02-18 17:30:56.180 | INFO     | __main__:<module>:23 - Image processor of type 'subtract' for image '['instanseg_cell', 'instanseg_nucleus']' has been created.


## Run ROIs Processing

In [6]:
# # Optional - to detect any problems with the provided sdata before running the pipeline
# for sd_path in path_list:
    
#     logger.info(f"Validating {sd_path.name}")

#     # get sdata
#     sdata = sd.read_zarr(sd_path)

#     # check that the pipeline can run on provide sdata
#     config.validate_pipeline(sdata)

2026-02-09 14:34:06.217 | INFO     | __main__:<module>:4 - Validating ROI_000.zarr
2026-02-09 14:34:06.751 | INFO     | plex_pipe.utils.config_schema:validate_pipeline:230 - ✅ Pipeline validation successful.
2026-02-09 14:34:06.751 | INFO     | __main__:<module>:4 - Validating ROI_001.zarr
2026-02-09 14:34:07.184 | INFO     | plex_pipe.utils.config_schema:validate_pipeline:230 - ✅ Pipeline validation successful.


In [6]:
for sd_path in path_list:
    
    logger.info(f"Processing {sd_path.name}")

    # get sdata
    sdata = sd.read_zarr(sd_path)

    # check that the pipeline can run on provide sdata
    config.validate_pipeline(sdata)

    # run builders of additional elements
    for builder_controller in builders_list:
        sdata = builder_controller.run(sdata)

2026-02-18 17:31:04.488 | INFO     | __main__:<module>:3 - Processing ROI_000.zarr
2026-02-18 17:31:04.940 | INFO     | plex_pipe.utils.config_schema:validate_pipeline:226 - ✅ Pipeline validation successful.
2026-02-18 17:31:04.942 | INFO     | plex_pipe.processors.controller:validate_resolution_present:85 - All channels have required resolution level: 0
2026-02-18 17:31:06.191 | INFO     | plex_pipe.processors.image_enhancers:run:74 - Applied normalization (percentiles 1.0–99.8) → [0.0, 130.0]
2026-02-18 17:31:06.194 | INFO     | plex_pipe.processors.controller:run:214 - New element(s) '['DAPI_norm']' have been created.
2026-02-18 17:31:06.368 | INFO     | plex_pipe.processors.controller:validate_resolution_present:85 - All channels have required resolution level: 0
2026-02-18 17:31:06.919 | INFO     | plex_pipe.processors.image_enhancers:run:74 - Applied normalization (percentiles 1.0–99.8) → [0.0, 45.0]
2026-02-18 17:31:06.921 | INFO     | plex_pipe.processors.controller:run:214 - N

d:\plex-pipe\.venv\Lib\site-packages\instanseg\utils\pytorch_utils.py:312: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:51.)
  intersection = torch.sparse.mm(onehot1, onehot2.T).to_dense()


2026-02-18 17:31:32.258 | INFO     | plex_pipe.processors.controller:run:214 - New element(s) '['instanseg_nucleus', 'instanseg_cell']' have been created.


d:\plex-pipe\.venv\Lib\site-packages\ome_zarr\writer.py:301: FutureWarning: Passing storage-related arguments via **kwargs is deprecated. Please use the 'zarr_store_kwargs' parameter instead. **kwargs will be removed in a future version.
  da_delayed = da.to_zarr(


2026-02-18 17:31:33.766 | INFO     | plex_pipe.processors.controller:run:235 - Mask 'instanseg_nucleus' has been saved to disk.


...

2026-02-18 17:32:04.564 | INFO     | plex_pipe.processors.controller:run:235 - Mask 'instanseg_cell' has been saved to disk.
2026-02-18 17:32:04.597 | INFO     | plex_pipe.processors.controller:validate_resolution_present:85 - All channels have required resolution level: 0
2026-02-18 17:32:11.235 | INFO     | plex_pipe.processors.controller:run:214 - New element(s) '['ring']' have been created.


d:\plex-pipe\.venv\Lib\site-packages\ome_zarr\writer.py:301: FutureWarning: Passing storage-related arguments via **kwargs is deprecated. Please use the 'zarr_store_kwargs' parameter instead. **kwargs will be removed in a future version.
  da_delayed = da.to_zarr(


2026-02-18 17:32:12.391 | INFO     | plex_pipe.processors.controller:run:235 - Mask 'ring' has been saved to disk.
2026-02-18 17:32:12.416 | INFO     | plex_pipe.processors.controller:validate_resolution_present:85 - All channels have required resolution level: 0
2026-02-18 17:32:12.859 | INFO     | plex_pipe.processors.controller:run:214 - New element(s) '['cytoplasm']' have been created.


d:\plex-pipe\.venv\Lib\site-packages\ome_zarr\writer.py:301: FutureWarning: Passing storage-related arguments via **kwargs is deprecated. Please use the 'zarr_store_kwargs' parameter instead. **kwargs will be removed in a future version.
  da_delayed = da.to_zarr(


2026-02-18 17:32:13.978 | INFO     | plex_pipe.processors.controller:run:235 - Mask 'cytoplasm' has been saved to disk.
